# Анализ данных для сервиса чтения книг по подписке.

## Описание проекта.

Цель проекта: анализ информации для создания ценностного предложения для нового продукта
Задачи:
Выяснение количества изданных книг с 2000 года;
Выяснение популярности изданных книг среди читателей (количество обзоров и средняя оценка);
Выяснение наиболее активного издательства, занимающегося публикацией объёмных книг (в нашем случае - 50 страниц и больше);
Определить самого популярного на данный момент автора (самый высокий средний рейтинг книг автора среди наиболее активных читателей, тех, кто оставил более 50 оценок);
Определить среднее количество отзывов от наиболее активных читателей, так как они влияют на выбор других читателей.

Источники информации:
База данных (пять таблиц), содержащая информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг и оценки читателей.

Структура проекта:
1. [Изучение данных.](#start)
2. [Исследовательский анализ.](#analysis)
3. [Вывод и рекомендации.](#outcome)

In [1]:
#Импортируем библиотеку и модуль для работы с данными в БД SQL data-analyst-final-project-db
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Устанавливаем параметры.
db_config = {'user': 'praktikum_student',
            'pwd': 'Sdf4$2;d-d30pp',
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432,
            'db': 'data-analyst-final-project-db'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                       db_config['pwd'],
                                                       db_config['host'],
                                                       db_config['port'],
                                                       db_config['db'])


In [3]:
#Сохраняем коннектор.
engine = create_engine(connection_string, connect_args = {'sslmode': 'require'})

## Изучение данных.
<a id="start"></a>

In [4]:
#Создаём функцию для более удобного вывода данных.
def pd_read_sql(query):
    return pd.io.sql.read_sql(query, con = engine)
#pd.io.sql.read_sql(query, con = engine)

In [5]:
#Исследуем данные в таблице books
books = '''
SELECT *
FROM books
LIMIT 10;
''' 

In [6]:
pd_read_sql(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [7]:
#Исследуем данные в таблице authors
authors = '''
SELECT *
FROM authors
LIMIT 10;
''' 

In [8]:
pd_read_sql(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [9]:
#Исследуем данные в таблице ratings
ratings = '''
SELECT *
FROM ratings
LIMIT 10;
''' 

In [10]:
pd_read_sql(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [11]:
#Исследуем данные в таблице reviews
reviews = '''
SELECT *
FROM reviews
LIMIT 10;
''' 

In [12]:
pd_read_sql(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [13]:
#Исследуем данные в таблице publishers
publishers = '''
SELECT *
FROM publishers
LIMIT 10;
''' 

In [14]:
pd_read_sql(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


## Исследовательский анализ.
<a id="analysis"></a>

In [15]:
#Считаем количество id изданных книг после 1 января 2000.
book_id_count = '''
SELECT COUNT(DISTINCT book_id) as books_number
FROM books
WHERE publication_date > '2000-01-01';
'''

In [16]:
pd_read_sql(book_id_count)

,books_number
0,819


In [17]:
#Считаем количество названий изданных книг после 1 января 2000.
title_count = '''
SELECT COUNT(DISTINCT title) as books_number--,
            --title
FROM books
WHERE publication_date > '2000-01-01'
--group by title
--having count(title) > 1
;
'''

In [18]:
pd_read_sql(title_count)

,books_number
0,818


После 1 января 2000 года издано 819 книг. Выпущено 818 наименований книг. Из них Memoirs of a Geisha - дважды.

In [19]:
#Считаем количество обзоров и среднюю оценку для книги.
assets_counts ='''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT re.review_id) as review_count,
       AVG(r.rating) as avg_rating
FROM books as b
LEFT JOIN reviews as re ON b.book_id = re.book_id
LEFT JOIN ratings as r ON b.book_id = r.book_id
GROUP BY b.book_id
ORDER BY avg_rating, review_count;
'''

In [20]:
pd_read_sql(assets_counts)

,book_id,title,review_count,avg_rating
0,303,Harvesting the Heart,2,1.50
1,316,His Excellency: George Washington,2,2.00
2,371,Junky,2,2.00
3,202,Drowning Ruth,3,2.00
4,915,The World Is Flat: A Brief History of the Twen...,3,2.25
...,...,...,...,...
995,76,Angels Fall,2,5.00
996,913,The Woman in Black,2,5.00
997,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
998,553,School's Out—Forever (Maximum Ride #2),3,5.00


Книги имеют рейтинг от 1.5 до 5 баллов. Количество ревью на каждую книгу от 0 до 7-ми.

In [21]:
#Выясняем самое активное издательство. С учётом книг в 50 и более стр.
best_publisher = '''
SELECT p.publisher_id,
       p.publisher
       
FROM publishers as p
LEFT JOIN books as b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY COUNT(DISTINCT b.book_id) DESC
LIMIT 1;
'''

In [22]:
pd_read_sql(best_publisher)

,publisher_id,publisher
0,212,Penguin Books


Самым активным издательством ожиданмо оказалось Penguin Books. Если вывести число изданных книг, увидим, что их 42.

In [23]:
#Выясняем самого популярного автора.C 50-ю и большим кол-м оценок
#Во внутреннем подзапросе выясняем средний рейтинг книг с количеством оценок 50 и больше.
#Во внешнем подзапросе считаем средний рейтинг всех книг для каждого автора.
author = '''
SELECT avg_rt.author
FROM
(
SELECT rt.author,
       SUM(rt.avg_rating) / COUNT(rt.title) as sum_rating
        
       
FROM
    (
    SELECT COUNT(r.rating_id) as num_rating,
           a.author,
           b.title,
           AVG(r.rating) as avg_rating
           
    FROM books as b
    LEFT JOIN ratings as r ON b.book_id = r.book_id
    LEFT JOIN authors as a ON a.author_id = b.author_id
    GROUP BY b.book_id, a.author_id, b.title
    HAVING COUNT(r.rating_id) > 49
    ORDER BY avg_rating DESC) as rt
GROUP BY  rt.author) as avg_rt
LIMIT 1
'''

In [24]:
pd_read_sql(author)

,author
0,J.K. Rowling/Mary GrandPré


Самый высокий средний рейтинг представленных на сервисе книг у американской писательницы Джоан Роулинг, работающей в жанрах: 
трагикомедия, криминальный роман и фэнтези.

In [25]:
num_review = '''
WITH
ratings AS(    
    SELECT r.username,
    COUNT(r.rating_id) as num_rating
    FROM ratings as r
    GROUP BY r.username
    HAVING COUNT(r.rating_id) > 50
    ),
    
reviews AS(
        SELECT COUNT(re.review_id) as num_review,
        re.username
        FROM reviews as re
        GROUP BY username
        )

SELECT ROUND(AVG (reviews.num_review)) as avg_number_of_reviews
FROM ratings LEFT JOIN reviews ON ratings.username = reviews.username
''' 

In [26]:
pd_read_sql(num_review)

,avg_number_of_reviews
0,24.0


Пользователи, поставившие более 50-ти оценок в среднем написали 24 (24.3333) отзыва.

## Выводы.
<a id="outcome"></a>

После 1 января 2000 года издано 819 книг. Выпущено 818 наименований книг. Из них Memoirs of a Geisha - дважды, что может свидетельствовать о её популярности. Следовательно приобретаемый сервис должен предлагать читателям не меньшее количество книг.

Книги имеют рейтинг от 1.5 до 5 баллов. Количество ревью на каждую книгу от 0 до 7-ми. Возможно, стоит выполнить отбор книг для сервиса, так, чтобы рейтинг был не ниже среднего. Отбор по количеству ревью нежелателен. Это может отсечь хорошие, но не самые массовые книги.

Самым активным издательством ожиданмо оказалось Penguin Books. Издательство опубликовало 42 книги. Следовательно именно на книги этого издательства стоит обратить особое внимание.

Самый высокий средний рейтинг книг у американской писательницы Джоан Роулинг, работающей в жанрах: 
трагикомедия, криминальный роман и фэнтези, как и в целом книги жанров, в которых она работает.

Активные читатели, поставившие более 50 оценок, написали в среднем по 24 ревью на прочитанные книги. Следовательно, читателям в сервисе должна быть предоставленна возможность писать отзывы, обсуждать прочитанные книги, и советовать новые книги друг-другу.